<a href="https://colab.research.google.com/github/alexmacharia/deep_learning_projects/blob/main/8_Pytorch_Fashion_MNIST_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import PIL
import numpy as np

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim
import torch.nn as nn

print("GPU available: {}".format(torch.cuda.is_available()))

GPU available: False


In [2]:
device = 'cpu'

In [3]:
data_transforms = {
    'train': transforms.Compose([
        # Transforms are executed in the order they are called
        # Some transforms return color image hence the need to convert image back to grayscale
        transforms.RandomAffine(degrees=10, translate=(0.05,0.05), shear=5),
        transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,)),

    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
    ])
}